In [1]:
import torch

import data
import utils
import info_recorder as ir
import data_loader as dl
import initializer as init
import trainer as tn

In [2]:
# https://chriskhanhtran.github.io/posts/cnn-sentence-classification/

In [3]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


There are 1 GPU(s) available.
Device name: NVIDIA GeForce GTX 1070


In [4]:
df, size_info, size_dict = data.get_df()

In [5]:
sourceCode_np = df.sourceCode.values
codeClass_np = df.classLabel.values

In [6]:
# Tokenize, build vocabulary, encode tokens
print("Tokenizing...\n")
tokenized_sourceCodes, ch2idx, max_len = utils.tokenize(sourceCode_np)
input_ids = utils.encode(tokenized_sourceCodes, ch2idx, max_len)

Tokenizing...



In [7]:
ir.record_ch2idx(ch2idx)

In [8]:
encoded_class2idx, class2idx, num_classes = utils.tokenize_encode_class(codeClass_np)

In [9]:
from sklearn.model_selection import train_test_split

train_inputs, val_inputs, train_labels, val_labels = train_test_split(
    input_ids, encoded_class2idx, test_size = 0.2, random_state = 43
)

In [10]:
# Load data to PyTorch DataLoader
train_dataloader, val_dataloader = dl.data_loader(train_inputs, val_inputs, train_labels, val_labels, batch_size=50)

In [11]:
# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('512chars/cnn')

In [12]:
# CNN-rand: Word vectors are randomly initialized.
tn.set_seed(42)
cnn_rand, optimizer = init.initilize_model(device=device,
                                           vocab_size=len(ch2idx),
                                           embed_dim=100,
                                           filter_sizes=[8, 8, 8, 8],
                                           num_filters=[100, 200, 200, 100],
                                           num_classes=len(class2idx),
                                           dropout=0.5,
                                           learning_rate=0.25,
                                           optimizerName="Adadelta",
                                           modelType="CNN")
                                        
# tn.train(device, cnn_rand, optimizer, train_dataloader, '01_4Layer_512data_8KS', writer, val_dataloader, epochs=20)

In [13]:
cnn_rand

CNN_NLP(
  (embedding): Embedding(97, 100, padding_idx=0, max_norm=5.0)
  (conv1d_list): ModuleList(
    (0): Conv1d(100, 100, kernel_size=(8,), stride=(1,))
    (1): Conv1d(100, 200, kernel_size=(8,), stride=(1,))
    (2): Conv1d(100, 200, kernel_size=(8,), stride=(1,))
    (3): Conv1d(100, 100, kernel_size=(8,), stride=(1,))
  )
  (fc): Linear(in_features=600, out_features=21, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [14]:
# # CNN-rand: Word vectors are randomly initialized.
# tn.set_seed(42)
# cnn_rand, optimizer = init.initilize_model(device=device,
#                                            vocab_size=len(ch2idx),
#                                            embed_dim=100,
#                                            filter_sizes=[8, 16, 16, 8],
#                                            num_filters=[100, 200, 200, 100],
#                                            num_classes=len(class2idx),
#                                            dropout=0.5,
#                                            learning_rate=0.25,
#                                            optimizerName="Adadelta",
#                                            modelType="CNN")
                                        
# tn.train(device, cnn_rand, optimizer, train_dataloader, '01_4Layer_512data_8+16KS', writer, val_dataloader, epochs=20)

In [15]:
# # CNN-rand: Word vectors are randomly initialized.
# tn.set_seed(42)
# cnn_rand, optimizer = init.initilize_model(device=device,
#                                            vocab_size=len(ch2idx),
#                                            embed_dim=100,
#                                            filter_sizes=[8, 16, 16, 8],
#                                            num_filters=[100, 200, 200, 100],
#                                            num_classes=len(class2idx),
#                                            dropout=0.65,
#                                            learning_rate=0.25,
#                                            optimizerName="Adadelta",
#                                            modelType="CNN")
                                        
# tn.train(device, cnn_rand, optimizer, train_dataloader, '01_4Layer_512data_65DP_8+16KS', writer, val_dataloader, epochs=20)